In [1]:
import pandas as pd
import numpy as np

import requests
import re
import io

In [ ]:
df = pd.read_csv('../../data/cleaned/wildfire.csv')

df['FireDiscoveryDateTime'] = pd.to_datetime(df['FireDiscoveryDateTime'], infer_datetime_format=True, errors = 'coerce')
df['ControlDateTime'] = pd.to_datetime(df['ControlDateTime'], infer_datetime_format=True, errors = 'coerce')

In [ ]:
def build_request_param(id, lat, long, start_date, end_date, result_df, base_uri):
    print(id)
    params = f"latitude={lat}&longitude={long}&start={start_date.strftime('%Y%m%d')}&end={end_date.strftime('%Y%m%d')}"
    base_uri += params

    res = requests.get(base_uri)

    split_text = res.text.split("-END HEADER-")

    if len(split_text) >= 2:
        response_df = pd.read_csv(io.StringIO(split_text[1]))
        # response_df['DATE'] = response_df['YEAR'].apply(str) + '-' + response_df['MO'].apply(str) + '-' + response_df['DY'].apply(str)
        # response_df.drop(columns=['YEAR', 'MO', 'DY'], inplace=True)
        response_df["LAT"] = lat
        response_df["LONG"] = long
        response_df["ID"] = id
        result_df = pd.concat([result_df, response_df])
        return result_df
        
    return result_df

In [ ]:
nasa_df = pd.DataFrame()
parameters = "parameters=T2M,T2M_MAX,QV2M,PRECTOTCORR,WS2M,WS2M_MAX,WS10M,WS10M_MAX,GWETTOP,GWETPROF"
base_uri = f"https://power.larc.nasa.gov/api/temporal/daily/point?time-standard=lst&header=true&format=csv&community=sb&{parameters}&"

for i in df.index:
    nasa_df = build_request_param(
        i,
        df.loc[i, "InitialLatitude"],
        df.loc[i, "InitialLongitude"],
        df.loc[i, "FireDiscoveryDateTime"],
        df.loc[i, "ControlDateTime"],
        nasa_df,
        base_uri,
    )

In [ ]:
nasa_df.to_csv("../../data/raw/nasa_weather.csv")
